<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/Co_training_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train1.head()

Mounted at /content/drive


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [2]:

train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  #print(i)
  train = train.append(i)

In [3]:

train.sort_values("Sentence", inplace = True)
print(len(train))

37711


In [4]:
new_train = train.drop_duplicates(subset ="Sentence")

In [5]:
train = new_train

In [6]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

array(['Invalid', 'Rule/Law/Holding', 'Facts', 'Analysis', 'Conclusion',
       'Issue'], dtype=object)

In [7]:
#Threshold= 0.98

In [8]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(4416, 7374)


In [10]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [11]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

classifier_rf = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
classifier_lgb = lgb.LGBMClassifier()

In [12]:
train = train.rename(columns={'Sentence':'text'})
train.head()

,Unnamed: 0,text,Target
1364,239,"""(I)n the First Amendment area 'government may...",Invalid
1833,185,"""... that nowhere in the statute was it stated...",Invalid
2431,415,"""Although a statute may be neither vague, over...",Rule/Law/Holding
2245,416,"""For even when pursuing a legitimate interest,...",Rule/Law/Holding
1561,25,"""If an indictment has been found or accusation...",Facts


In [13]:

unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
unlabel.head()


,Unnamed: 0,text,Complete
0,0,No motion for rehearing filed.,1
1,1,— Relator is under accusation of killing her h...,1
2,2,From an order of the District Judge of the 39t...,1
3,3,Relator and her husband had been married for a...,1
4,4,The evidence indicates that their family relat...,1


In [14]:
del unlabel['Complete']

In [15]:
del unlabel['Unnamed: 0']

In [16]:
unlabel.head()

,text
0,No motion for rehearing filed.
1,— Relator is under accusation of killing her h...
2,From an order of the District Judge of the 39t...
3,Relator and her husband had been married for a...
4,The evidence indicates that their family relat...


In [17]:
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


In [18]:
from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
unlabel.head()

,text
0,motion rehearing filed
1,relator accusation killing husband virgil clark
2,order district judge 39th judicial district re...
3,relator husband married number year
4,evidence indicates family relation always plea...


In [20]:

len(unlabel)

537703

In [21]:
# unlabel_1 = unlabel.loc[:100000]
# print("length of unlabel_1",len(unlabel_1))
# unlabel_2 = unlabel.loc[100001:200000]
# print("length of unlabel_2",len(unlabel_2))
# unlabel_3 = unlabel.loc[200001:300000]
# print("length of unlabel_3",len(unlabel_3))
# unlabel_4 = unlabel.loc[300001:400000]
# print("length of unlabel_4",len(unlabel_4))
# unlabel_5 = unlabel.loc[400001:]
# print("length of unlabel_5",len(unlabel_5))

In [22]:
unlabel_1 = unlabel.loc[:100]
unlabel_1.head()

,text
0,motion rehearing filed
1,relator accusation killing husband virgil clark
2,order district judge 39th judicial district re...
3,relator husband married number year
4,evidence indicates family relation always plea...


In [23]:

# X_train_whole, x_val_whole, Y_train_whole, y_val_whole = train_test_split(X_tfidf,train['Target'],test_size=0.1,random_state=42)

In [24]:
# X_tfidf.shape

In [25]:
# classifier_rf.fit(X_tfidf, train['Target'])

In [26]:
# test_pred = classifier_rf.predict(t_p)
# accuracy_test = accuracy_score(test['Target'],test_pred)
# score_test = f1_score(test['Target'],test_pred, average='weighted')
# print("Test accuracy test",accuracy_test )
# print("Test f1 score test",score_test )

In [27]:
import numpy as np
from sklearn.datasets import make_classification

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



#X, y = make_classification(random_state=1)

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.20,random_state=42)

X_train_whole = X_tfidf
Y_train_whole = train['Target']
#X_train_whole, x_val_whole, Y_train_whole, y_val_whole = train_test_split(X_tfidf,train['Target'],test_size=0.1,random_state=42)

classifier_rf.fit(X_train, Y_train)
classifier_lgb.fit(X_train, Y_train)

val_pred_rf = classifier_rf.predict(x_val)
val_pred_lgb = classifier_lgb.predict(x_val)


Accuracy_score_rf = accuracy_score(y_val,val_pred_rf)
print("Accuracy_score_rf ",Accuracy_score_rf)
Accuracy_score_lgb = accuracy_score(y_val,val_pred_lgb)
print("Accuracy_score_lgb ",Accuracy_score_lgb)

classification_report_test_rf = classification_report(y_val,val_pred_rf)
print("classification report ",classification_report_test_rf)
classification_report_test_lgb = classification_report(y_val,val_pred_lgb)
print("classification report ",classification_report_test_lgb)


f1_score_rf = f1_score(y_val,val_pred_rf,average='weighted')
print(" f1 score test ",f1_score_rf )
f1_score_lgb = f1_score(y_val,val_pred_lgb, average='weighted')
print(" f1 score test ",f1_score_lgb )


if Accuracy_score_rf > Accuracy_score_lgb:
  B = classifier_rf
else:
  B = classifier_lgb

B.fit(X_train_whole,Y_train_whole)
# class_x_un1 = tfidf_vect.transform(unlabel_1['text'])
# pseudo_pred = B.predict(class_x_un1)

class_x_un1 = tfidf_vect.transform(unlabel_1['text'])

class_pred_unlabel_1 = B.predict(class_x_un1)
unlabel_1['Target']=class_pred_unlabel_1
frame_1 = [train,unlabel_1]
train_1 = pd.concat(frame_1)
print(len(train_1))




# print('Validation Accuracy_score: %f' % Accuracy_score)
# test_pred = classifier.predict(t_p)
# acc_test = accuracy_score(test_pred,test['Target'])
# print('Accuracy test data', acc_test)
# matrix = confusion_matrix(y_pred, y_val)
# print(matrix)


Accuracy_score_rf  0.6334841628959276
Accuracy_score_lgb  0.6470588235294118
classification report                    precision    recall  f1-score   support

        Analysis       0.51      0.26      0.35       140
      Conclusion       0.58      0.36      0.44        42
           Facts       0.66      0.90      0.76       448
         Invalid       0.57      0.50      0.53       130
           Issue       0.90      0.33      0.49        54
Rule/Law/Holding       0.62      0.34      0.44        70

        accuracy                           0.63       884
       macro avg       0.64      0.45      0.50       884
    weighted avg       0.63      0.63      0.60       884

classification report                    precision    recall  f1-score   support

        Analysis       0.48      0.40      0.44       140
      Conclusion       0.57      0.40      0.47        42
           Facts       0.69      0.91      0.79       448
         Invalid       0.63      0.33      0.43       130
   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
test_pred = B.predict(t_p)
accuracy_test = accuracy_score(test['Target'],test_pred)
print("Test accuracy test",accuracy_test )
classification_report_test = classification_report(test['Target'],test_pred)
print("classification report ",classification_report_test)

Test accuracy test 0.6410748560460653
classification report                    precision    recall  f1-score   support

        Analysis       0.43      0.27      0.33        77
      Conclusion       0.71      0.65      0.68        26
           Facts       0.67      0.88      0.76       267
         Invalid       0.55      0.39      0.45        83
           Issue       0.92      0.32      0.48        34
Rule/Law/Holding       0.62      0.53      0.57        34

        accuracy                           0.64       521
       macro avg       0.65      0.51      0.55       521
    weighted avg       0.63      0.64      0.62       521



In [30]:
len(unlabel)

537703

In [32]:
length_list = range(100,500,100)

In [36]:
j=0

for i in length_list:
  print(i)
  print(j)
  unlabel_chunk = unlabel[j:i]
  j=i
  print("length",len(unlabel_chunk))
  #function call with different unlabel chunk

100
0
length 100
200
100
length 100
300
200
length 100
400
300
length 100
